In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient

from sklearn.decomposition import PCA
from numpy.random import default_rng

# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
# import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load Motor neuron table

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')
pre_to_mn_df.shape
pre_to_mn_df.sum(axis=0).sum()

In [ ]:
mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor_med_venU',
    'tarsus_depressor_noid',
    ]
pre_to_mn_df.head()

new_idx = utils.sort_segment_fcn_index(pre_to_mn_df.columns)
pre_to_mn_df = pre_to_mn_df[new_idx]

pre_to_mn_df.head()

In [ ]:
%config InlineBackend.figure_formats = ['png']
%matplotlib inline

# Just pool connectivity

## Flexors

In [ ]:
All = slice(None)
femur_df_local = pre_to_mn_df.loc[('local',All,All,All),('L','Leg','femur',All,All,All,All)]
# femur_df_all = pre_to_mn_df.loc[('local'),('L','Leg','femur',All,All,All,All)]

# mft_df = femur_df_local.loc[('main_tibia_flexor',All,All),mpool_dict['main_tibia_flexor_muscle']]
mft_df = femur_df_local.loc[('local','main_tibia_flexor',All,All),mpool_dict['aux_main_tibia_flexor']]
# mft_df = mft_df.iloc[0:50,:]

# just change the rank label on that one MN
clms = mft_df.columns.to_frame()
clms.loc[('L','Leg','femur','flex','main_tibia_flexor','0')].loc[[False,True],'rank']='1'
# clms.loc[('L','Leg','femur','flex','main_tibia_flexor','0',All),:].loc[[False,True],'rank']
mft_df.columns = pd.MultiIndex.from_frame(clms)
mft_df

ext_df = femur_df_local.loc[('local','tibia_extensor',All,All),mpool_dict['tibia_extensor']]
ext_df


# Leg modules: flexors

In [ ]:
from sklearn.decomposition import PCA
from numpy.random import default_rng

In [ ]:
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(mft_df.to_numpy())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

pca_decomp = PCA(n_components=None,
          copy=True, 
          whiten=False, 
          svd_solver='full', 
          tol=0.0, 
          iterated_power='auto')
mft_pca = pca_decomp.fit(mft_df)
mu = np.mean(mft_df, axis=0)
mu

# nComp = 5
# Xhat = np.dot(mft_pca.transform(mft_df)[:,:nComp], mft_pca.components_[:nComp,:])
# Xhat += mu

A0 = mft_pca.transform(mft_df)

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(A0)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'projection',)
plt.show()

# fig = plt.figure(1, figsize = [6,6])
# plt.plot(A0[0,:])
# plt.plot(A0[1,:])
# plt.plot(A0[2,:])

fig = plt.figure(1, figsize = [6,6])
plt.plot(mft_pca.explained_variance_/mft_pca.explained_variance_.sum())
A0var = np.var(A0,axis=0)
A0var
plt.plot(A0var/A0var.sum())
plt.ylabel('% variance')
plt.xlabel('PC')
plt.title('Tibia Flexors (9 MNs)')

mft_pca.components_.shape
V = mft_pca.components_

fig,ax = plt.subplots(1,1, figsize = [6,6])
ax = sns.heatmap(V.transpose())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'dictionary transform of local_df',)
plt.show()


(V[:,0]**2).sum()
# So, the test is whether AV gives back the original matrix
Y = np.matmul(A0,V)

fig = plt.figure(1, figsize = [6,6])
Y += mu.to_numpy()
ax = sns.heatmap(Y)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

mft_pca.explained_variance_/mft_pca.explained_variance_.sum()

In [ ]:
fig = plt.figure(1, figsize = [6,6])
plt.plot(V[0,:])
plt.plot(V[1,:])
plt.plot(V[2,:])

plt.plot(mu.to_numpy())


## Add preMNs together and project onto components

In [ ]:
# Create a matrix of N=1000 pairs
from numpy.random import default_rng

N = 1000
k = 2

rng = default_rng()
n = mft_df.shape[0]

fig = plt.figure(1, figsize = [6,6])

sumN = 15 # number of rows to sum
A_var_mat = np.full((sumN-1,len(A0var)),fill_value=0,dtype=float)
A_var_mat[0,:] = A0var
plt.plot(A0var/A0var.sum())

q = 0
for k in range(sumN-1)[1:]:
    rnd_rows = rng.integers(0,n-1,(N,k+1)) # start with 2 and go to sumN

    n_k_mat = np.full((N,mft_df.shape[1]),fill_value=0,dtype=float)
    for r in range(n_k_mat.shape[0]):
        n_k_mat[r,:] = mft_df.iloc[rnd_rows[r,:],:].sum(axis=0)

    A = mft_pca.transform(n_k_mat)
    
    A_var = np.var(A,axis=0)

    plt.plot(A_var/A_var.sum())
    A_var_mat[k,:] = A_var
    q = q+1


### Conclusion: no change
The original pcs continue to capture variance as more preMNs are added together. That's likely because the 2nd 3rd etc pcs are capturing variation along a different axis

## PCA sums of ~35 neurons
Try PCA on a matrix of simulated sums. Get the major components, then project individuals and sums onto those pcs.

In [ ]:
# Create a matrix of N=1000 pairs
from numpy.random import default_rng

rng = default_rng()
N = 2500
n = mft_df.shape[0]
k = np.floor(N/2).astype(int)
rnd_rows = rng.integers(0,n-1,(N,k+1))
n_k_mat = np.full((N,mft_df.shape[1]),fill_value=0,dtype=float)
for r in range(n_k_mat.shape[0]):
    n_k_mat[r,:] = mft_df.iloc[rnd_rows[r,:],:].sum(axis=0)

pca_decomp = PCA(n_components=None,
          copy=True, 
          whiten=False, 
          svd_solver='full', 
          tol=0.0, 
          iterated_power='auto')
n_k_pca = pca_decomp.fit(n_k_mat)

In [ ]:
V = n_k_pca.components_

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(V.transpose())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'dictionary transform of local_df',)
plt.show()

In [ ]:
# Now try to project into the n choose k space. The other components should capture a fair bit of the noise

fig = plt.figure(1, figsize = [6,6])

sumN = 35 # number of rows to sum
A_var_mat = np.full((sumN-1,len(A0var)),fill_value=0,dtype=float)
A_var_mat[0,:] = A0var
plt.plot(A0var/A0var.sum())

for k in range(sumN-1)[1:]:
    rnd_rows = rng.integers(0,n-1,(N,k+1)) # start with 2 and go to sumN

    n_k_mat = np.full((N,mft_df.shape[1]),fill_value=0,dtype=float)
    for r in range(n_k_mat.shape[0]):
        n_k_mat[r,:] = mft_df.iloc[rnd_rows[r,:],:].sum(axis=0)

    A = n_k_pca.transform(n_k_mat)
    
    A_var = np.var(A,axis=0)

    plt.plot(A_var/A_var.sum())
    A_var_mat[k,:] = A_var



In [ ]:
fig = plt.figure(1, figsize = [6,6])
plt.plot(A_var_mat[:,0]/A_var_mat.sum(axis=1))

### Conclusion: No more variance onto first component!
Huh! The variance captured by the first components is similar to when using the original pca space

# Leg modules: Extensors

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(ext_df.to_numpy())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

pca_decomp = PCA(n_components=None,
          copy=True, 
          whiten=False, 
          svd_solver='full', 
          tol=0.0, 
          iterated_power='auto')
ext_pca = pca_decomp.fit(ext_df)
mu = np.mean(ext_df, axis=0)
mu

# nComp = 5
# Xhat = np.dot(mft_pca.transform(mft_df)[:,:nComp], mft_pca.components_[:nComp,:])
# Xhat += mu

A0 = ext_pca.transform(ext_df)

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(A0)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'projection',)
plt.show()

fig = plt.figure(1, figsize = [6,6])
plt.plot(ext_pca.explained_variance_/ext_pca.explained_variance_.sum())
A0var = np.var(A0,axis=0)
A0var
plt.plot(A0var/A0var.sum())
plt.ylabel('% variance')
plt.xlabel('PC')
plt.title('Tibia Extensors (FETi, SETi)')


ext_pca.components_.shape
V = ext_pca.components_

fig,ax = plt.subplots(1,1, figsize = [6,6])
ax = sns.heatmap(V.transpose())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'dictionary transform of local_df',)
plt.show()


(V[:,0]**2).sum()
# So, the test is whether AV gives back the original matrix
Y = np.matmul(A0,V)

fig = plt.figure(1, figsize = [6,6])
Y += mu.to_numpy()
ax = sns.heatmap(Y)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

In [ ]:
ext_pca.explained_variance_/ext_pca.explained_variance_.sum()

## Add preMNs together and project onto components

In [ ]:
# Create a matrix of N=1000 pairs
from numpy.random import default_rng

N = 2500
k = 2

rng = default_rng()
n = ext_df.shape[0]

fig = plt.figure(1, figsize = [6,6])

sumN = 15 # number of rows to sum
A_var_mat = np.full((sumN-1,len(A0var)),fill_value=0,dtype=float)
A_var_mat[0,:] = A0var
plt.plot(A0var/A0var.sum())

for k in range(sumN-1)[1:]:
    rnd_rows = rng.integers(0,n-1,(N,k+1)) # start with 2 and go to sumN

    n_k_mat = np.full((N,ext_df.shape[1]),fill_value=0,dtype=float)
    for r in range(n_k_mat.shape[0]):
        n_k_mat[r,:] = ext_df.iloc[rnd_rows[r,:],:].sum(axis=0)

    A = ext_pca.transform(n_k_mat)
    
    A_var = np.var(A,axis=0)

    plt.plot(A_var/A_var.sum())
    A_var_mat[k,:] = A_var

# All leg modules

In [ ]:
import utils
mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension', 
    'tergotrochanter',          
    'extracoxal_trochanter_depressor',
    'trochanter_extensor',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor_wtarsus',
    'auxiliary_tibia_flexor_B_wtarsus',
    'auxiliary_tibia_flexor_E_wtarsus',
    # 'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    # 'auxiliary_tibia_flexor_B',
    # 'auxiliary_tibia_flexor_E',
    'ltm',
    'ltm_tiny_small',
    'ltm_tibia',
    'ltm_femur',
    # 'tarsus_depressor_med_venU',
    # 'tarsus_depressor_noid',
    ]

pref_pool_dict = {
    'thorax_swing': 'thorax_swing',
    'thorax_stance': 'thorax_stance',
    'trochanter_extension': 'trochanter_extension',
    'tergotrochanter': 'trochanter_extension',
    'extracoxal_trochanter_depressor' : 'trochanter_extension',
    'trochanter_extensor': 'trochanter_extension',
    'trochanter_flexion': 'trochanter_flexion',
    'femur_reductor':  'femur_reductor',
    'tibia_extensor': 'tibia_extensor',
    'main_tibia_flexor_wtarsus': 'main_tibia_flexor',
    'auxiliary_tibia_flexor_B_wtarsus': 'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E_wtarsus':  'auxiliary_tibia_flexor_E',
    'ltm': 'ltm',
    'ltm_tiny_small': 'ltm',
    'ltm_tibia': 'ltm',
    'ltm_femur': 'ltm',
    #'tarsus_depressor_med_venU': ,
}
fname = './figpanels/PCA_leg_modules_pct_tr_ext_ltm_breakout.svg'
# pre_to_mn_df.head()
# mpool_dict

In [ ]:
import utils
mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension', 
    # 'tergotrochanter',          
    # 'extracoxal_trochanter_depressor',
    # 'trochanter_extensor',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor_wtarsus',
    'auxiliary_tibia_flexor_B_wtarsus',
    'auxiliary_tibia_flexor_E_wtarsus',
    # 'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    # 'auxiliary_tibia_flexor_B',
    # 'auxiliary_tibia_flexor_E',
    'ltm',
    #'ltm_tiny_small',
    #'ltm_tibia',
    #'ltm_femur',
    # 'tarsus_depressor_med_venU',
    # 'tarsus_depressor_noid',
    ]

pref_pool_dict = {
    'thorax_swing': 'thorax_swing',
    'thorax_stance': 'thorax_stance',
    'trochanter_extension': 'trochanter_extension',
    'tergotrochanter': 'trochanter_extension',
    'extracoxal_trochanter_depressor' : 'trochanter_extension',
    'trochanter_extensor': 'trochanter_extension',
    'trochanter_flexion': 'trochanter_flexion',
    'femur_reductor':  'femur_reductor',
    'tibia_extensor': 'tibia_extensor',
    'main_tibia_flexor_wtarsus': 'main_tibia_flexor',
    'auxiliary_tibia_flexor_B_wtarsus': 'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E_wtarsus':  'auxiliary_tibia_flexor_E',
    'ltm': 'ltm',
    'ltm_tiny_small': 'ltm',
    'ltm_tibia': 'ltm',
    'ltm_femur': 'ltm',
    #'tarsus_depressor_med_venU': ,
}
fname = './figpanels/PCA_leg_modules_pct.svg'
# pre_to_mn_df.head()
# mpool_dict

var_frac_dict = {}
pca_decomp = PCA(n_components=None,
        copy=True, 
        whiten=False, 
        svd_solver='full', 
        tol=0.0, 
        iterated_power='auto')

for pool in pool_keys:
    syn = local_df.loc[pref_pool_dict[pool],mpool_dict[pool]]
    print(syn.shape)
    mu = np.mean(syn, axis=0)

    syn_pca = pca_decomp.fit(syn)
    A0 = syn_pca.transform(syn)
    var_frac_dict[pool] = syn_pca.explained_variance_/syn_pca.explained_variance_.sum()


In [ ]:
local_df = pre_to_mn_df.loc['local',:]
# local_df.head()

In [ ]:
var_frac_dict = {}
pca_decomp = PCA(n_components=None,
        copy=True, 
        whiten=False, 
        svd_solver='full', 
        tol=0.0, 
        iterated_power='auto')

for pool in pool_keys:
    syn = local_df.loc[pref_pool_dict[pool],mpool_dict[pool]]
    print(syn.shape)
    mu = np.mean(syn, axis=0)

    syn_pca = pca_decomp.fit(syn)
    A0 = syn_pca.transform(syn)
    var_frac_dict[pool] = syn_pca.explained_variance_/syn_pca.explained_variance_.sum()

fig, ax = plt.subplots( 1, 1, figsize=(6,6))
clrs = sns.color_palette()
clrs = [
    '#555555',
    '#888888',
    '#A0A0A0',
    '#BFBFBF',
    '#DFDFDF',
    '#E0E0E0',
    '#E7E7E7',
    '#EBEBEB',
    '#EFEFEF',
    '#F2F2F2',
    '#F4F4F4',
    '#F8F8F8',
]

m = 0
pc1 = []
pc2 = []
for pool in pool_keys:
    pc_frac = var_frac_dict[pool]
    bottom = 0.
    pc1+=[pc_frac[0]]
    pc2+=[pc_frac[1]]
    for pc_fr,clr in zip(pc_frac,clrs):
        p = ax.bar(m, pc_fr, 0.8, bottom=bottom,color=clr)
        bottom += pc_fr
    m+=1

x = [i for i in range(len(pool_keys))]
ax.plot(x,pc1,color='#000000')
ax.plot(x,pc2,color=clrs[1])
ax.plot(x,np.array(pc2)/np.array(pc1))
ax.set_xticks(x)
ax.set_xticklabels(pool_keys)
plt.xticks(rotation=90)

fig.savefig(fname,format='svg')
fig.savefig('./figpanels/PCA_leg_modules_pct.svg',format='svg')

In [ ]:
# Plot the first 3 PCs for Tibia flexors

# mft_df = femur_df_local.loc[('local','main_tibia_flexor',All,All),mpool_dict['aux_main_tibia_flexor']]

pref_pool_dict['aux_main_tibia_flexor_wtarsus'] = 'main_tibia_flexor'

syn = local_df.loc[pref_pool_dict['aux_main_tibia_flexor_wtarsus'],mpool_dict['aux_main_tibia_flexor_wtarsus']]
print(syn.shape)
mu = np.mean(syn, axis=0)

syn_pca = pca_decomp.fit(syn)
A0 = syn_pca.transform(syn)

fig, ax = plt.subplots( 1, 1, figsize=(6,6))
ax.plot(syn_pca.components_[0,:])
ax.plot(syn_pca.components_[1,:])
ax.plot(syn_pca.components_[2,:])
fig.savefig('./figpanels/tibiaflexA_wtarsus_PCs.svg',format='svg')


In [ ]:
pre_to_mn_df.loc[:,mpool_dict['tergotrochanter']]

# Steering synergies: Load

In [ ]:
df = pd.read_pickle('./data/wingMN_adj_0314.pkl')

In [ ]:
df

In [ ]:
modules = df.columns.get_level_values('MN_module').unique()
preferred_module = df.index.get_level_values('preferred_module').unique()
preferred_module

In [ ]:
modules = ['DLM', 'DVM', 'ten', 'inc', 'dec', 'dow', 'unk']#,'hg2', ]

In [ ]:
module_dict = {
    'DLM':'DLM',
    'DVM':'DVM',
    'ten':'tension',
    'inc':'syn4',
    'dec':'syn3',
    'dow':'syn1',
    'unk':'syn2',
    'hg2':'syn5',    
}
var_frac_dict = {}

In [ ]:
df.loc[:,'ten']

In [ ]:
All = slice(None)
var_frac_dict = {}
pca_decomp = PCA(n_components=None,
        copy=True, 
        whiten=False, 
        svd_solver='full', 
        tol=0.0, 
        iterated_power='auto')

for module in modules:
    syn = df.loc[(All,module_dict[module],All,'local'),module]
    print(syn.shape)
    mu = np.mean(syn, axis=0)

    syn_pca = pca_decomp.fit(syn)
    A0 = syn_pca.transform(syn)
    var_frac_dict[module] = syn_pca.explained_variance_/syn_pca.explained_variance_.sum()


fig, ax = plt.subplots( 1, 1, figsize=(6,6))
clrs = [
    '#555555',
    '#888888',
    '#A0A0A0',
    '#BFBFBF',
    '#DFDFDF',
    '#E0E0E0',
    '#E7E7E7',
    '#EBEBEB',
    '#EFEFEF',
    '#F2F2F2',
    '#F4F4F4',
    '#F8F8F8',
]
m = 0
pc1 = []
pc2 = []
for module in modules:
    pc_frac = var_frac_dict[module]
    pc1+=[pc_frac[0]]
    try:
        pc2+=[pc_frac[1]]
    except IndexError:
        pc2+=[0]
    bottom = 0.
    for pc_fr,clr in zip(pc_frac,clrs):
        p = ax.bar(m, pc_fr, 0.8, bottom=bottom,color=clr)
        bottom += pc_fr
    m+=1

x = [i for i in range(len(modules))]
ax.plot(x,pc1)
ax.plot(x,np.array(pc2)/np.array(pc1))
ax.set_xticks(x)
ax.set_xticklabels(modules)
plt.xticks(rotation=90)

fig.savefig('./figpanels/PCA_wing_modules_pct.svg',format='svg')


In [ ]:
df.loc[]
# module_dict['inc']

In [ ]:
# Plot the first 3 PCs for Tibia flexors

# mft_df = femur_df_local.loc[('local','main_tibia_flexor',All,All),mpool_dict['aux_main_tibia_flexor']]

# pref_pool_dict['inc'] = 'main_tibia_flexor'
syn = df.loc[(All,module_dict['inc'],All,'local'),'inc']
# syn = df.loc[module_dict['inc'],'inc']
print(syn.shape)

mu = np.mean(syn, axis=0)

syn_pca = pca_decomp.fit(syn)
A0 = syn_pca.transform(syn)

fig, ax = plt.subplots( 1, 1, figsize=(6,6))
ax.plot(syn_pca.components_[0,:])
ax.plot(syn_pca.components_[1,:])
ax.plot(syn_pca.components_[2,:])
fig.savefig('./figpanels/WBA_inc_PCs.svg',format='svg')


In [ ]:

mu = np.mean(syn, axis=0)

syn_pca = pca_decomp.fit(syn)
A0 = syn_pca.transform(syn)

fig, ax = plt.subplots( 1, 1, figsize=(6,6))
ax.plot(syn_pca.components_[0,:])
ax.plot(syn_pca.components_[1,:])
ax.plot(syn_pca.components_[2,:])
fig.savefig('./figpanels/tibiaflexA_wtarsus_PCs.svg',format='svg')


In [ ]:
modules

In [ ]:
fig = plt.figure(1, figsize = [6,6])
plt.plot(syn4_pca.explained_variance_/syn4_pca.explained_variance_.sum())
A0var = np.var(A0,axis=0)
A0var
plt.plot(A0var/A0var.sum())
plt.ylabel('% variance')
plt.xlabel('PC')
plt.title('Syn 4 (tpn,iii1,iii3)')

print(syn4_pca.components_.shape)
V = syn4_pca.components_

fig,ax = plt.subplots(1,1, figsize = [6,6])
ax = sns.heatmap(V.transpose())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'dictionary transform of local_df',)
plt.show()


(V[:,0]**2).sum()
# So, the test is whether AV gives back the original matrix
Y = np.matmul(A0,V)

fig = plt.figure(1, figsize = [6,6])
Y += mu.to_numpy()
ax = sns.heatmap(Y)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

syn4_pca.explained_variance_/syn4_pca.explained_variance_.sum()

# Steering synergies: syn4

In [ ]:
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(syn4.to_numpy())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

pca_decomp = PCA(n_components=None,
          copy=True, 
          whiten=False, 
          svd_solver='full', 
          tol=0.0, 
          iterated_power='auto')
syn4_pca = pca_decomp.fit(syn4)
mu = np.mean(syn4, axis=0)
mu

# nComp = 5
# Xhat = np.dot(mft_pca.transform(mft_df)[:,:nComp], mft_pca.components_[:nComp,:])
# Xhat += mu

A0 = syn4_pca.transform(syn4)

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(A0)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'projection',)
plt.show()

# fig = plt.figure(1, figsize = [6,6])
# plt.plot(A0[0,:])
# plt.plot(A0[1,:])
# plt.plot(A0[2,:])

fig = plt.figure(1, figsize = [6,6])
plt.plot(syn4_pca.explained_variance_/syn4_pca.explained_variance_.sum())
A0var = np.var(A0,axis=0)
A0var
plt.plot(A0var/A0var.sum())
plt.ylabel('% variance')
plt.xlabel('PC')
plt.title('Syn 4 (tpn,iii1,iii3)')

print(syn4_pca.components_.shape)
V = syn4_pca.components_

fig,ax = plt.subplots(1,1, figsize = [6,6])
ax = sns.heatmap(V.transpose())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'dictionary transform of local_df',)
plt.show()


(V[:,0]**2).sum()
# So, the test is whether AV gives back the original matrix
Y = np.matmul(A0,V)

fig = plt.figure(1, figsize = [6,6])
Y += mu.to_numpy()
ax = sns.heatmap(Y)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

syn4_pca.explained_variance_/syn4_pca.explained_variance_.sum()

## add preMNs together, project onto pca space

In [ ]:
# Create a matrix of N=1000 pairs
from numpy.random import default_rng

N = 2500
k = 2

rng = default_rng()
n = syn4.shape[0]

fig = plt.figure(1, figsize = [6,6])

sumN = 15 # number of rows to sum
A_var_mat = np.full((sumN-1,len(A0var)),fill_value=0,dtype=float)
A_var_mat[0,:] = A0var
plt.plot(A0var/A0var.sum())

for k in range(sumN-1)[1:]:
    rnd_rows = rng.integers(0,n-1,(N,k+1)) # start with 2 and go to sumN

    n_k_mat = np.full((N,syn4.shape[1]),fill_value=0,dtype=float)
    for r in range(n_k_mat.shape[0]):
        n_k_mat[r,:] = syn4.iloc[rnd_rows[r,:],:].sum(axis=0)

    A = syn4_pca.transform(n_k_mat)
    
    A_var = np.var(A,axis=0)

    plt.plot(A_var/A_var.sum())
    A_var_mat[k,:] = A_var


# Steering synergies: Syn 3

In [ ]:
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(syn3.to_numpy())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

pca_decomp = PCA(n_components=None,
          copy=True, 
          whiten=False, 
          svd_solver='full', 
          tol=0.0, 
          iterated_power='auto')
syn3_pca = pca_decomp.fit(syn3)
mu = np.mean(syn3, axis=0)
mu

# nComp = 5
# Xhat = np.dot(mft_pca.transform(mft_df)[:,:nComp], mft_pca.components_[:nComp,:])
# Xhat += mu

A0 = syn3_pca.transform(syn3)

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(A0)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'projection',)
plt.show()

# fig = plt.figure(1, figsize = [6,6])
# plt.plot(A0[0,:])
# plt.plot(A0[1,:])
# plt.plot(A0[2,:])

fig = plt.figure(1, figsize = [6,6])
plt.plot(syn3_pca.explained_variance_/syn3_pca.explained_variance_.sum())
A0var = np.var(A0,axis=0)
A0var
plt.plot(A0var/A0var.sum())
plt.ylabel('% variance')
plt.xlabel('PC')
plt.title('Syn 3 (i2_u,i1,b3_u)')

print(syn3_pca.components_.shape)
V = syn3_pca.components_

fig,ax = plt.subplots(1,1, figsize = [6,6])
ax = sns.heatmap(V.transpose())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'dictionary transform of local_df',)
plt.show()


(V[:,0]**2).sum()
# So, the test is whether AV gives back the original matrix
Y = np.matmul(A0,V)

fig = plt.figure(1, figsize = [6,6])
Y += mu.to_numpy()
ax = sns.heatmap(Y)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

syn3_pca.explained_variance_/syn3_pca.explained_variance_.sum()

## add preMNs together, project onto pca space

In [ ]:
# Create a matrix of N=1000 pairs
from numpy.random import default_rng

N = 2500
k = 2

rng = default_rng()
n = syn3.shape[0]

fig = plt.figure(1, figsize = [6,6])

sumN = 15 # number of rows to sum
A_var_mat = np.full((sumN-1,len(A0var)),fill_value=0,dtype=float)
A_var_mat[0,:] = A0var
plt.plot(A0var/A0var.sum())

for k in range(sumN-1)[1:]:
    rnd_rows = rng.integers(0,n-1,(N,k+1)) # start with 2 and go to sumN

    n_k_mat = np.full((N,syn3.shape[1]),fill_value=0,dtype=float)
    for r in range(n_k_mat.shape[0]):
        n_k_mat[r,:] = syn3.iloc[rnd_rows[r,:],:].sum(axis=0)

    A = syn3_pca.transform(n_k_mat)
    
    A_var = np.var(A,axis=0)

    plt.plot(A_var/A_var.sum())
    A_var_mat[k,:] = A_var


# Steering synergies: Syn 1 (b1, b2)

In [ ]:
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(syn1.to_numpy())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

pca_decomp = PCA(n_components=None,
          copy=True, 
          whiten=False, 
          svd_solver='full', 
          tol=0.0, 
          iterated_power='auto')
syn1_pca = pca_decomp.fit(syn1)
mu = np.mean(syn1, axis=0)
mu

# nComp = 5
# Xhat = np.dot(mft_pca.transform(mft_df)[:,:nComp], mft_pca.components_[:nComp,:])
# Xhat += mu

A0 = syn1_pca.transform(syn1)

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(A0)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'projection',)
plt.show()

# fig = plt.figure(1, figsize = [6,6])
# plt.plot(A0[0,:])
# plt.plot(A0[1,:])
# plt.plot(A0[2,:])

fig = plt.figure(1, figsize = [6,6])
plt.plot(syn1_pca.explained_variance_/syn1_pca.explained_variance_.sum())
A0var = np.var(A0,axis=0)
A0var
plt.plot(A0var/A0var.sum())
plt.ylabel('% variance')
plt.xlabel('PC')
plt.title('Syn 1 (b1, b2)')

print(syn1_pca.components_.shape)
V = syn1_pca.components_

fig,ax = plt.subplots(1,1, figsize = [6,6])
ax = sns.heatmap(V.transpose())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'dictionary transform of local_df',)
plt.show()


(V[:,0]**2).sum()
# So, the test is whether AV gives back the original matrix
Y = np.matmul(A0,V)

fig = plt.figure(1, figsize = [6,6])
Y += mu.to_numpy()
ax = sns.heatmap(Y)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

syn1_pca.explained_variance_/syn1_pca.explained_variance_.sum()

## add preMNs together, project onto pca space

# Now try a normalized version
Just for kicks and understanding

In [ ]:
# First normalize by the sum of the rows
ratiodf = mft_df.divide(mft_df.sum(axis=1),axis='index')
ratiodf.replace([np.inf, -np.inf], np.nan, inplace=True)
ratiodf = ratiodf.dropna()

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(ratiodf.to_numpy())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()

pca_decomp_n = PCA(n_components=None,
          copy=True, 
          whiten=False, 
          svd_solver='full', 
          tol=0.0, 
          iterated_power='auto')
mft_pca_n = pca_decomp_n.fit(ratiodf)
mu = np.mean(ratiodf, axis=0)

# nComp = 5
# Xhat = np.dot(mft_pca.transform(mft_df)[:,:nComp], mft_pca.components_[:nComp,:])
# Xhat += mu

A0 = mft_pca_n.transform(ratiodf)

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(A0)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'projection',)
plt.show()

# fig = plt.figure(1, figsize = [6,6])
# plt.plot(A0[0,:])
# plt.plot(A0[1,:])
# plt.plot(A0[2,:])

fig = plt.figure(1, figsize = [6,6])
plt.plot(mft_pca_n.explained_variance_/mft_pca_n.explained_variance_.sum())
A0var = np.var(A0,axis=0)
A0var
plt.plot(A0var/A0var.sum())

print(mft_pca_n.components_.shape)
V = mft_pca_n.components_

fig,ax = plt.subplots(1,1, figsize = [6,6])
ax = sns.heatmap(V.transpose())
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'dictionary transform of local_df',)
plt.show()


(V[:,0]**2).sum()
# So, the test is whether AV gives back the original matrix
Y = np.matmul(A0,V)

fig = plt.figure(1, figsize = [6,6])
Y += mu.to_numpy()
ax = sns.heatmap(Y)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()


## add preMNs together, project onto pca_n space

In [ ]:
# Create a matrix of N=1000 pairs
from numpy.random import default_rng

N = 2500
k = 2

rng = default_rng()
n = ratiodf.shape[0]

fig = plt.figure(1, figsize = [6,6])

sumN = 15 # number of rows to sum
A_var_mat = np.full((sumN-1,len(A0var)),fill_value=0,dtype=float)
A_var_mat[0,:] = A0var
plt.plot(A0var/A0var.sum())

for k in range(sumN-1)[1:]:
    rnd_rows = rng.integers(0,n-1,(N,k+1)) # start with 2 and go to sumN

    n_k_mat = np.full((N,ratiodf.shape[1]),fill_value=0,dtype=float)
    for r in range(n_k_mat.shape[0]):
        n_k_mat[r,:] = ratiodf.iloc[rnd_rows[r,:],:].sum(axis=0)

    A = mft_pca_n.transform(n_k_mat)
    
    A_var = np.var(A,axis=0)

    plt.plot(A_var/A_var.sum())
    A_var_mat[k,:] = A_var

# Try SVD instead

In [ ]:
x_bar = mft_df.mean(axis=1)
A = mft_df.subtract(mft_df.mean(axis=1),axis='index').to_numpy()

In [ ]:
fig = plt.figure(1, figsize = [6,6])

ax = sns.heatmap(A)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'log 10 # of synapses',)
plt.show()


In [ ]:
u, s, vh = np.linalg.svd(A)
u.shape, s.shape, vh.shape

In [ ]:
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(vh)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'dictionary transform of local_df',)
plt.show()

In [ ]:
s

In [ ]:
SIG = np.diag(s)

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(SIG)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'dictionary transform of local_df',)
plt.show()

In [ ]:
A_pca = np.matmul(A,vh.T)
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(A_pca)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'projection',)
plt.show()

A_svh=  np.matmul(SIG,A_pca)

In [ ]:
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(A_pca)
cbar = ax.collections[0].colorbar
cbar.set_label(label = 'projection',)
plt.show()

In [ ]:
pre_to_mn_df.columns.get_level_values('segID').to_list()